# Demo to show how to use the Asset Search API 

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
from os import getenv
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(getenv("SERENITY_CONFIG_ID", None))

In [ ]:
import pandas as pd

from serenity_sdk.renderers.asset_search.result_views import (
    make_option_grid_df,
    make_asset_symbol_df
)
from serenity_sdk.renderers.shared.utils import make_df_from_list

from serenity_types.refdata.asset import AssetSearchRequest, AssetType
from serenity_sdk.renderers.shared.simple_widgets import display_in_hbox

api = connect_widget.api

In [ ]:
# default display columns
default_display_cols = ["asset_id", "symbol"]

## TOKENS

In [ ]:
token_df = make_df_from_list(api.refdata().search_assets(
    request=AssetSearchRequest(asset_types=[AssetType.TOKEN])
))
token_symbol_df = make_asset_symbol_df(token_df)
display_in_hbox(token_symbol_df[default_display_cols].style)

In [ ]:
# create a native symbol -> asset ID map for later use
native_symbol_map = token_symbol_df["asset_id"]
native_symbol_map.head(2)

## Exchanges
Get supported exchangs

In [ ]:
exchange_map = pd.Series(api.refdata().get_exchanges())
exchange_map

## Perpetuals

Find perpetuals by exchange id

In [ ]:
# set the exchange id
exchange_id = exchange_map["bitmex"] # binance, bitmex, deribit, okex

perpetual_df = make_df_from_list(api.refdata().search_assets(
    request=AssetSearchRequest(
        asset_types=[AssetType.PERPETUAL], exchange_ids=[exchange_id]
    )
))
perpetual_symbol_df = make_asset_symbol_df(perpetual_df)
display_in_hbox(
    perpetual_symbol_df[default_display_cols + ["payoff_type", "contract_size"]].style
)

## Futures

Find futures by exchange id

In [ ]:
exchange_id = exchange_map["okex"] # binance, bitmex, deribit, okex
futures_df = make_df_from_list(api.refdata().search_assets(
    request=AssetSearchRequest(
        asset_types=[AssetType.FUTURE], exchange_ids=[exchange_id]
    )
))
futures_symbol_df = make_asset_symbol_df(futures_df)
display_in_hbox(
    futures_symbol_df[default_display_cols + ["payoff_type", "contract_size"]].style
)

## Options

Find options by exchange & underlier id

In [ ]:
# currently, only BTC & ETH are supported

exchange_id = exchange_map["deribit"]  # deribit, bybit, okex
underlier_id = native_symbol_map["BTC"]  # BTC, ETH

option_df =  make_df_from_list(api.refdata().search_assets(
    request=AssetSearchRequest(
        asset_types=[AssetType.LISTED_OPTION],
        underlier_assets=[underlier_id],
        exchange_ids=[exchange_id],
    )
))
option_grid_df = make_option_grid_df(option_df)
display_in_hbox(option_grid_df.style, height="600px")

In [ ]:
option_symbol_df = make_asset_symbol_df(
    option_df, ["expiry_datetime", "strike_price", "option_type"]
)
print(f"number of options: {len(option_df)}")
display_in_hbox(option_df[default_display_cols + ["contract_size"]].style, height="600px")

## Describe Assets

Given asset id (uuid), get asset details

In [ ]:
asset_ids = option_df["asset_id"].to_list()[:3]
asset_ids

In [ ]:
asset_description_df = make_df_from_list(api.refdata().describe_assets(asset_ids))
display_in_hbox(asset_description_df.style)

# END